In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from Bio import SeqIO
from collections import Counter

In [2]:
test = 'KK*TGVVGVYK*ALNLSRGEP'
print(test.find('*'))
print(test[:test.find('*')])

2
KK


In [18]:
def trim_seq(seq):
    index = seq.find('*') 
    if index >= 0:
        return seq[:index]
    else:
        return seq

In [20]:
path = 'tables/adt5111_data_files_s1.csv'
df1 = pd.read_csv(path)
df1['aa_seq_trimmed'] = df1.aa_seq.apply(trim_seq)
df1 = df1.rename(columns={'aa_seq_trimmed':'aa_seq',
                          'aa_seq':'aa_seq_stop'})
df1 = df1[(~df1['aa_seq'].isna())&(df1['aa_seq']!='')].copy(deep=True)
df1

,aa_seq_stop,dataset,seed_bh,fitness,sigma,aa_seq
0,NKHELWQRIGAVVRYTLNLL,NNK1,0,0.057809,0.344230,NKHELWQRIGAVVRYTLNLL
1,KKKRRLSSWYSHPVPSSGLL,NNK1,0,0.487563,1.041771,KKKRRLSSWYSHPVPSSGLL
2,KKKYTEHISFTALHKLFTRR,NNK1,0,0.188875,0.921912,KKKYTEHISFTALHKLFTRR
3,KKNIWPLYSLRVRLYELAIS,NNK1,0,-0.965352,1.677212,KKNIWPLYSLRVRLYELAIS
4,KKTGVVGVYK*ALNLSRGEP,NNK1,0,-1.256191,1.075006,KKTGVVGVYK
...,...,...,...,...,...,...
131780,FF*VMFYISWMVYPLGISDG,NNK4,0,NaN,NaN,FF
131781,FF*FGFNCLYLVTYCHYVAT,NNK4,0,NaN,NaN,FF
131782,FFY*VCFNGV*NECKIVCES,NNK4,0,NaN,NaN,FFY
131783,FFYLFDSCLGLFFYKDYLSS,NNK4,0,NaN,NaN,FFYLFDSCLGLFFYKDYLSS


In [33]:
path = 'tables/adt5111_data_files_s2.csv'
df2 = pd.read_csv(path)
df2 = df2[(~df2['aa_seq'].isna())&(df2['aa_seq']!='')].copy(deep=True)
df2

,aa_seq,seed_bh,dataset,train_test
0,IWTKSCDCVCSCGGYQGCYR,0,NNK1,train
1,ILKSFLD,0,NNK1,train
2,WTMLDMFLFS,0,NNK1,train
3,CLQCKCYNNCVQLRVTVGCM,1,NNK1,train
4,WTWILPMVRTIIYQY,0,NNK1,train
...,...,...,...,...
111687,CCDSLFKDRCFSWYTCCASF,0,NNK3,test
111688,TFFDMLVVIRRILVLCRDGS,0,NNK3,test
111689,VTRLANVGVNYFFDAVGTDP,0,NNK3,test
111690,FFTFKKFYYFRCNFVLCFFC,1,NNK3,test


In [34]:
35456+37578+38893

111927

# Checks

In [45]:
# short seq are duplicated
df1.groupby('aa_seq').dataset.nunique().sort_values()

aa_seq
RLQDNRDVATGH            1
RLRAWKVPCL              1
RLRAQSHCAVAAGRNMEPWA    1
RLR                     1
RLQVRLSLLRWFRNYGLTIA    1
                       ..
E                       4
SV                      4
KM                      4
AF                      4
FV                      4
Name: dataset, Length: 121663, dtype: int64

In [50]:
# do the labels match?....
seq2label = dict(zip(df2.aa_seq,df2.seed_bh))
df1c = df1.copy(deep=True)
df1c['seed_bh_2'] = df1c.aa_seq.apply(lambda x: seq2label[x] if x in seq2label else '')
df1c['seed_bh_match'] = df1c['seed_bh'] == df1c['seed_bh_2']
df1c

,aa_seq_stop,dataset,seed_bh,fitness,sigma,aa_seq,seed_bh_2,seed_bh_match
0,NKHELWQRIGAVVRYTLNLL,NNK1,0,0.057809,0.344230,NKHELWQRIGAVVRYTLNLL,0,True
1,KKKRRLSSWYSHPVPSSGLL,NNK1,0,0.487563,1.041771,KKKRRLSSWYSHPVPSSGLL,0,True
2,KKKYTEHISFTALHKLFTRR,NNK1,0,0.188875,0.921912,KKKYTEHISFTALHKLFTRR,0,True
3,KKNIWPLYSLRVRLYELAIS,NNK1,0,-0.965352,1.677212,KKNIWPLYSLRVRLYELAIS,0,True
4,KKTGVVGVYK*ALNLSRGEP,NNK1,0,-1.256191,1.075006,KKTGVVGVYK,0,True
...,...,...,...,...,...,...,...,...
131780,FF*VMFYISWMVYPLGISDG,NNK4,0,NaN,NaN,FF,0,True
131781,FF*FGFNCLYLVTYCHYVAT,NNK4,0,NaN,NaN,FF,0,True
131782,FFY*VCFNGV*NECKIVCES,NNK4,0,NaN,NaN,FFY,0,True
131783,FFYLFDSCLGLFFYKDYLSS,NNK4,0,NaN,NaN,FFYLFDSCLGLFFYKDYLSS,,False


In [52]:
df1c.groupby(['dataset','seed_bh_match']).size()

dataset  seed_bh_match
NNK1     False             2074
         True             37149
NNK2     False             2496
         True             39217
NNK3     False             1698
         True             41519
NNK4     False             6933
         True               699
dtype: int64

In [55]:
df1c[~df1c['seed_bh_match']].head()

,aa_seq_stop,dataset,seed_bh,fitness,sigma,aa_seq,seed_bh_2,seed_bh_match
35,KKCIYKANRFFFIHWSLLFF,NNK1,0,0.737330,0.299932,KKCIYKANRFFFIHWSLLFF,1,False
67,KNARNISICMMCIPGPQYFY,NNK1,0,0.834789,0.324349,KNARNISICMMCIPGPQYFY,1,False
97,KNFFRKVMRKGNLRFS*KPT,NNK1,0,1.559518,0.618989,KNFFRKVMRKGNLRFS,1,False
98,KTT*AYYRTRYFHLSNGMRC,NNK1,0,1.251235,0.529024,KTT,1,False
104,KTHMVWRVERESIREVHVRG,NNK1,0,0.983073,0.402145,KTHMVWRVERESIREVHVRG,1,False


The labels don't match! Probably because the sequences appear across the datasets with different labels

In [79]:
df1[df1['dataset']=='NNK4']

,aa_seq_stop,dataset,seed_bh,fitness,sigma,aa_seq
124153,KKQETSSRIMRLVGL*FGGK,NNK4,0,-0.602943,1.713057,KKQETSSRIMRLVGL
124154,KKL*LESVGDDLMSYDCAV*,NNK4,0,-0.315489,1.041303,KKL
124155,KKFINSSR*RTIITNNAFFE,NNK4,1,0.787592,0.224586,KKFINSSR
124156,KNYMIVINDAFWFW*CFGMF,NNK4,1,2.182734,0.090781,KNYMIVINDAFWFW
124157,KNYCTFLNGLRDFSNSCWVS,NNK4,1,1.614852,0.110646,KNYCTFLNGLRDFSNSCWVS
...,...,...,...,...,...,...
131780,FF*VMFYISWMVYPLGISDG,NNK4,0,NaN,NaN,FF
131781,FF*FGFNCLYLVTYCHYVAT,NNK4,0,NaN,NaN,FF
131782,FFY*VCFNGV*NECKIVCES,NNK4,0,NaN,NaN,FFY
131783,FFYLFDSCLGLFFYKDYLSS,NNK4,0,NaN,NaN,FFYLFDSCLGLFFYKDYLSS


In [84]:
dup = []
overlap = []
for seq,group in df1.groupby('aa_seq'):
    if len(group) == 1:
        continue
    dup.append(group)
    if 'NNK4' not in group.dataset:
        continue
    elif group.dataset.nunique() <= 1:
        continue
    overlap.append(group)
dup = pd.concat(dup)
print(len(dup))
if overlap:
    overlap = pd.concat(overlap)
    overlap

11619


# Plot lengths

In [57]:
temp = df1.copy(deep=True)
temp['length'] = temp.aa_seq.apply(lambda x: len(x))
temp['length'].max()                              

20

# Prep dataset

Make the following decisions
- Ignore that there is overlap between NNK datasets and keep it as is
- Use the labels from S2 (which were what was used for training CANYA)
- Add the labels for NNK4 from S1, truncate the sequences at stops

In [23]:
data_dir = '/novo/projects/departments/cdd/molecular_ai/mlbp/data/fibrillation_data'

In [35]:
def combine_dups(df):
    data = []
    for seq,group in df.groupby('aa_seq',sort=False):
        # display(group)
        if len(group) == 1:
            data.append(group.iloc[0])
        else:
            agg_frac = group.seed_bh.mean()
            if agg_frac == 0.5:
                print('skipping')
                continue
            else:
                group['seed_bh'] = round(agg_frac)
                data.append(group.iloc[0])
        # print(data)
        # break
    return pd.concat(data,axis=1).transpose()

combine_dups(df1[df1['dataset']=='NNK4'].copy(deep=True))

skipping
skipping
skipping


,aa_seq_stop,dataset,seed_bh,fitness,sigma,aa_seq
124153,KKQETSSRIMRLVGL*FGGK,NNK4,0,-0.602943,1.713057,KKQETSSRIMRLVGL
124154,KKL*LESVGDDLMSYDCAV*,NNK4,0,-0.315489,1.041303,KKL
124155,KKFINSSR*RTIITNNAFFE,NNK4,1,0.787592,0.224586,KKFINSSR
124156,KNYMIVINDAFWFW*CFGMF,NNK4,1,2.182734,0.090781,KNYMIVINDAFWFW
124157,KNYCTFLNGLRDFSNSCWVS,NNK4,1,1.614852,0.110646,KNYCTFLNGLRDFSNSCWVS
...,...,...,...,...,...,...
131778,FFVEGESEAINKCLVLQK*L,NNK4,0,NaN,NaN,FFVEGESEAINKCLVLQK
131779,FF*ECSRPDRNIEYIDACNR,NNK4,0,NaN,NaN,FF
131782,FFY*VCFNGV*NECKIVCES,NNK4,0,NaN,NaN,FFY
131783,FFYLFDSCLGLFFYKDYLSS,NNK4,0,NaN,NaN,FFYLFDSCLGLFFYKDYLSS


In [36]:
temp = df1[df1['dataset']=='NNK4']
print(len(temp))
temp1 = combine_dups(temp)
print(len(temp1))
test_df = temp1[['aa_seq','dataset','seed_bh']].copy(deep=True)
test_df['fold'] = -1
test_df

7400
skipping
skipping
skipping
7039


,aa_seq,dataset,seed_bh,fold
124153,KKQETSSRIMRLVGL,NNK4,0,-1
124154,KKL,NNK4,0,-1
124155,KKFINSSR,NNK4,1,-1
124156,KNYMIVINDAFWFW,NNK4,1,-1
124157,KNYCTFLNGLRDFSNSCWVS,NNK4,1,-1
...,...,...,...,...
131778,FFVEGESEAINKCLVLQK,NNK4,0,-1
131779,FF,NNK4,0,-1
131782,FFY,NNK4,0,-1
131783,FFYLFDSCLGLFFYKDYLSS,NNK4,0,-1


In [37]:
temp3 = df2[df2['train_test']=='train'].drop(columns='train_test')
print(len(temp3))
temp3

100730


,aa_seq,seed_bh,dataset
0,IWTKSCDCVCSCGGYQGCYR,0,NNK1
1,ILKSFLD,0,NNK1
2,WTMLDMFLFS,0,NNK1
3,CLQCKCYNNCVQLRVTVGCM,1,NNK1
4,WTWILPMVRTIIYQY,0,NNK1
...,...,...,...
107877,FEWLWIFWYNLFWLWGNYSI,0,NNK3
107878,KDYLLSSYCEECHDHSSSEL,0,NNK3
107879,DFWRTIEGKMSLALI,0,NNK3
107880,TWVVYFVVRMCVQGSVYSMA,0,NNK3


In [38]:
# shuffle and split into 5 folds
# Section 2.1.1
canya_df = temp3.sample(frac=1,random_state=42).reset_index(drop=True)

# split the dataset into 5 batches
n_batches = 5
n_subset = math.ceil(len(canya_df)/n_batches)
canya_df['fold'] = 1+canya_df.index//n_subset
display(canya_df.groupby('fold').size())
canya_df.head()

fold
1    20146
2    20146
3    20146
4    20146
5    20146
dtype: int64

,aa_seq,seed_bh,dataset,fold
0,VHMNRSGSAALEFGMLWEYI,0,NNK3,1
1,QLVRQMTKGVG,0,NNK3,1
2,WFIRARFTLGKRNELREVI,0,NNK1,1
3,CCAVLFRGVRGMVRCNWNFD,0,NNK3,1
4,MLCGINGMMSSTLVCGRAII,0,NNK2,1


In [39]:
# create a new column for each fold
for i in sorted(list(canya_df.fold.unique())):
    col_name = f'data_split_fold{i}'
    canya_df[col_name] = canya_df['fold'].apply(lambda x: 'val' if x == i else 'train')
    print(f'Split: {i}')
    print('Validation indices:',canya_df[canya_df[col_name]=='val'].index)
canya_df.head()

Split: 1
Validation indices: Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       20136, 20137, 20138, 20139, 20140, 20141, 20142, 20143, 20144, 20145],
      dtype='int64', length=20146)
Split: 2
Validation indices: Index([20146, 20147, 20148, 20149, 20150, 20151, 20152, 20153, 20154, 20155,
       ...
       40282, 40283, 40284, 40285, 40286, 40287, 40288, 40289, 40290, 40291],
      dtype='int64', length=20146)
Split: 3
Validation indices: Index([40292, 40293, 40294, 40295, 40296, 40297, 40298, 40299, 40300, 40301,
       ...
       60428, 60429, 60430, 60431, 60432, 60433, 60434, 60435, 60436, 60437],
      dtype='int64', length=20146)
Split: 4
Validation indices: Index([60438, 60439, 60440, 60441, 60442, 60443, 60444, 60445, 60446, 60447,
       ...
       80574, 80575, 80576, 80577, 80578, 80579, 80580, 80581, 80582, 80583],
      dtype='int64', length=20146)
Split: 5
Validation indices: Index([ 80584,  80585,  80586,  80587,  80588,  805

,aa_seq,seed_bh,dataset,fold,data_split_fold1,data_split_fold2,data_split_fold3,data_split_fold4,data_split_fold5
0,VHMNRSGSAALEFGMLWEYI,0,NNK3,1,val,train,train,train,train
1,QLVRQMTKGVG,0,NNK3,1,val,train,train,train,train
2,WFIRARFTLGKRNELREVI,0,NNK1,1,val,train,train,train,train
3,CCAVLFRGVRGMVRCNWNFD,0,NNK3,1,val,train,train,train,train
4,MLCGINGMMSSTLVCGRAII,0,NNK2,1,val,train,train,train,train


In [40]:
# add test data
for col in sorted(x for x in canya_df.columns if 'data_split_fold' in x):
    test_df[col] = 'test'
test_df

,aa_seq,dataset,seed_bh,fold,data_split_fold1,data_split_fold2,data_split_fold3,data_split_fold4,data_split_fold5
124153,KKQETSSRIMRLVGL,NNK4,0,-1,test,test,test,test,test
124154,KKL,NNK4,0,-1,test,test,test,test,test
124155,KKFINSSR,NNK4,1,-1,test,test,test,test,test
124156,KNYMIVINDAFWFW,NNK4,1,-1,test,test,test,test,test
124157,KNYCTFLNGLRDFSNSCWVS,NNK4,1,-1,test,test,test,test,test
...,...,...,...,...,...,...,...,...,...
131778,FFVEGESEAINKCLVLQK,NNK4,0,-1,test,test,test,test,test
131779,FF,NNK4,0,-1,test,test,test,test,test
131782,FFY,NNK4,0,-1,test,test,test,test,test
131783,FFYLFDSCLGLFFYKDYLSS,NNK4,0,-1,test,test,test,test,test


In [49]:
concat_df = pd.concat([canya_df,test_df],ignore_index=True)
concat_df.info()
concat_df = concat_df.rename(columns={'aa_seq':'sequence',
                          'seed_bh':'value_bool'})
concat_df.to_csv(f'datasets/sbxw_fibrillation_peptide_canyaNNK1to3-5fold_train-val_canyaNNK4-test.csv')
concat_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107769 entries, 0 to 107768
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   aa_seq            107769 non-null  object
 1   seed_bh           107769 non-null  object
 2   dataset           107769 non-null  object
 3   fold              107769 non-null  int64 
 4   data_split_fold1  107769 non-null  object
 5   data_split_fold2  107769 non-null  object
 6   data_split_fold3  107769 non-null  object
 7   data_split_fold4  107769 non-null  object
 8   data_split_fold5  107769 non-null  object
dtypes: int64(1), object(8)
memory usage: 7.4+ MB


,sequence,value_bool,dataset,fold,data_split_fold1,data_split_fold2,data_split_fold3,data_split_fold4,data_split_fold5
0,VHMNRSGSAALEFGMLWEYI,0,NNK3,1,val,train,train,train,train
1,QLVRQMTKGVG,0,NNK3,1,val,train,train,train,train
2,WFIRARFTLGKRNELREVI,0,NNK1,1,val,train,train,train,train
3,CCAVLFRGVRGMVRCNWNFD,0,NNK3,1,val,train,train,train,train
4,MLCGINGMMSSTLVCGRAII,0,NNK2,1,val,train,train,train,train
...,...,...,...,...,...,...,...,...,...
107764,FFVEGESEAINKCLVLQK,0,NNK4,-1,test,test,test,test,test
107765,FF,0,NNK4,-1,test,test,test,test,test
107766,FFY,0,NNK4,-1,test,test,test,test,test
107767,FFYLFDSCLGLFFYKDYLSS,0,NNK4,-1,test,test,test,test,test


In [51]:
path = '/novo/projects/departments/cdd/molecular_ai/mlbp/data/fibrillation_data/sbxw_fibrillation_peptide_canyaNNK1to3-5fold_train-val_canyaNNK4-test.csv'
df = pd.read_csv(path,index_col=0)
df

,sequence,value_bool,dataset,fold,data_split_fold1,data_split_fold2,data_split_fold3,data_split_fold4,data_split_fold5
0,VHMNRSGSAALEFGMLWEYI,0,NNK3,1,val,train,train,train,train
1,QLVRQMTKGVG,0,NNK3,1,val,train,train,train,train
2,WFIRARFTLGKRNELREVI,0,NNK1,1,val,train,train,train,train
3,CCAVLFRGVRGMVRCNWNFD,0,NNK3,1,val,train,train,train,train
4,MLCGINGMMSSTLVCGRAII,0,NNK2,1,val,train,train,train,train
...,...,...,...,...,...,...,...,...,...
107764,FFVEGESEAINKCLVLQK,0,NNK4,-1,test,test,test,test,test
107765,FF,0,NNK4,-1,test,test,test,test,test
107766,FFY,0,NNK4,-1,test,test,test,test,test
107767,FFYLFDSCLGLFFYKDYLSS,0,NNK4,-1,test,test,test,test,test


In [52]:
df[df['fold']==-1]

,sequence,value_bool,dataset,fold,data_split_fold1,data_split_fold2,data_split_fold3,data_split_fold4,data_split_fold5
100730,KKQETSSRIMRLVGL,0,NNK4,-1,test,test,test,test,test
100731,KKL,0,NNK4,-1,test,test,test,test,test
100732,KKFINSSR,1,NNK4,-1,test,test,test,test,test
100733,KNYMIVINDAFWFW,1,NNK4,-1,test,test,test,test,test
100734,KNYCTFLNGLRDFSNSCWVS,1,NNK4,-1,test,test,test,test,test
...,...,...,...,...,...,...,...,...,...
107764,FFVEGESEAINKCLVLQK,0,NNK4,-1,test,test,test,test,test
107765,FF,0,NNK4,-1,test,test,test,test,test
107766,FFY,0,NNK4,-1,test,test,test,test,test
107767,FFYLFDSCLGLFFYKDYLSS,0,NNK4,-1,test,test,test,test,test


In [44]:
df['aa_seq'].sort_values()

102065                       A
16312                       AA
53834     AAACAGIKFLLTFTFGFRTW
53551     AAAFSSGGKLFHAGCWRRMM
901                   AAAFYWLN
                  ...         
68962     YYYVTCSILPLFYFEKGLAL
67824     YYYVWNFGGWIGFRNVVSIR
100627    YYYYDRCMRPVHSGKGVKLW
57036             YYYYPNGDCEKK
106542    YYYYYVDTYDSTLANLLIYY
Name: aa_seq, Length: 107769, dtype: object